# 轻松构建 PyTorch 生成对抗网络(GAN)

互联网环境里有很多公开的数据集，对于机器学习的工程和科研很有帮助，比如算法学习和效果评价。我们将使用 MNIST 这个手写字体数据集训练模型，最终生成逼真的『手写』字体效果图样。


### Environment setup
Upgrade packages

In [ ]:
!pip install --upgrade pip -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install --upgrade sagemaker awscli boto3 pandas -i https://pypi.tuna.tsinghua.edu.cn/simple

Following commands for ```SageMaker Studio``` only

In [ ]:
!pip uninstall -y tqdm

import os
os.chdir('/root/ml-on-aws/byos-pytorch-gan')

!pwd

### 数据准备

PyTorch 框架的 torchvision.datasets 包提供了 MNIST 数据集，您可以通过如下指令下载 MNIST 数据集到本地备用。
(为方便国内读者使用，将 QMNIST 数据集替换为 MNIST 数据集，经测试训练数据的下载速度提高了很多)

In [ ]:
from torchvision import datasets

dataroot = './data'

trainset = datasets.MNIST(root=dataroot, train=True, download=True)
testset = datasets.MNIST(root=dataroot, train=False, download=True)

Amazon SageMaker 为您创建了一个默认的 Amazon S3 桶，用来存取机器学习工作流程中可能需要的各种文件和数据。 我们可以通过 SageMaker SDK 中 sagemaker.session.Session 类的 default_bucket 方法获得这个桶的名字。

In [ ]:
from sagemaker.session import Session

sess = Session()

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = sess.default_bucket()

# Location to save your custom code in tar.gz format.
s3_custom_code_upload_location = f's3://{bucket}/customcode/byos-pytorch-gan'

# Location where results of model training are saved.
s3_model_artifacts_location = f's3://{bucket}/artifacts/'

SageMaker SDK 提供了操作 Amazon S3 服务的包和类，其中 S3Downloader 类用于访问或下载 S3 里的对象，而 S3Uploader 则用于将本地文件上传至 S3。您将已经下载的数据上传至 Amazon S3，供模型训练使用。模型训练过程不要从互联网下载数据，避免通过互联网获取训练数据的产生的网络延迟，同时也规避了因直接访问互联网对模型训练可能产生的安全风险。


In [ ]:
import os
from sagemaker.s3 import S3Uploader as s3up

s3_data_location = s3up.upload(os.path.join(dataroot, "MNIST"), f"s3://{bucket}/data/mnist")

### 训练执行




通过 sagemaker.get_execution_role() 方法，当前笔记本可以得到预先分配给笔记本实例的角色，这个角色将被用来获取训练用的资源，比如下载训练用框架镜像、分配 Amazon EC2 计算资源等等。

In [ ]:
from sagemaker import get_execution_role

# IAM execution role that gives SageMaker access to resources in your AWS account.
# We can use the SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

训练模型用的超参数可以在笔记本里定义，实现与算法代码的分离，在创建训练任务时传入超参数，与训练任务动态结合。

In [ ]:
import json

hps = {
         'seed': 0,
         'learning-rate': 0.0002,
         'epochs': 15,
         'dataset': 'mnist',
         'pin-memory': 1,
         'beta1': 0.5,
         'nc': 1,
         'nz': 100,
         'ngf': 28,
         'ndf': 28,
         'batch-size': 64,
         'sample-interval': 100,
         'log-interval': 20,
     }


str_hps = json.dumps(hps, indent = 4)
print(str_hps)

sagemaker.pytorch 包里的 ```PyTorch``` 类是基于 PyTorch 框架的模型拟合器，可以用来创建、执行训练任务，还可以对训练完的模型进行部署。参数列表中， ``train_instance_type`` 用来指定CPU或者GPU实例类型，训练脚本和包括模型代码所在的目录通过 ``source_dir`` 指定，训练脚本文件名必须通过 ``entry_point`` 明确定义。这些参数将和其余参数一起被传递给训练任务，他们决定了训练任务的运行环境和模型训练时参数。

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(role=role,
                        entry_point='train.py',
                        source_dir='networks/DCGAN',
                        output_path=s3_model_artifacts_location,
                        code_location=s3_custom_code_upload_location,
                        train_instance_count=1,
                        train_instance_type='ml.p3.2xlarge',
                        train_use_spot_instances=True,
                        train_max_wait=86400,
                        framework_version='1.4.0',
                        py_version='py3',
                        hyperparameters=hps,
                   )

请特别注意 ``train_use_spot_instances`` 参数，``True`` 值代表您希望优先使用 SPOT 实例。由于机器学习训练工作通常需要大量计算资源长时间运行，善用 SPOT 可以帮助您实现有效的成本控制，SPOT 实例价格可能是按需实例价格的 20% 到 60%，依据选择实例类型、区域、时间不同实际价格有所不同。 

您已经创建了 PyTorch 对象，下面可以用它来拟合预先存在 Amazon S3 上的数据了。下面的指令将执行训练任务，训练数据将以名为 **MNIST** 的输入通道的方式导入训练环境。训练开始执行过程中，Amazon S3 上的训练数据将被下载到模型训练环境的本地文件系统，训练脚本 ```train.py``` 将从本地磁盘加载数据进行训练。

In [ ]:
# Start training
estimator.fit({"MNIST": s3_data_location}, wait=False)

根据您选择的训练实例不同，训练过程中可能持续几十分钟到几个小时不等。建议设置 ``wait`` 参数为 ``False`` ，这个选项将使笔记本与训练任务分离，在训练时间长、训练日志多的场景下，可以避免笔记本上下文因为网络中断或者会话超时而丢失。训练任务脱离笔记本后，输出将暂时不可见，可以执行如下代码，笔记本将获取并载入此前的训练回话，

In [ ]:
%%time
from sagemaker.estimator import Estimator

# Attaching previous training session
training_job_name = estimator.latest_training_job.name
attached_estimator = Estimator.attach(training_job_name)

由于的模型设计考虑到了GPU对训练加速的能力，所以用GPU实例训练会比CPU实例快一些，例如，p3.2xlarge 实例大概需要15分钟左右，而 c5.xlarge 实例则可能需要6小时以上。目前模型不支持分布、并行训练，所以多实例、多CPU/GPU并不会带来更多的训练速度提升。

训练完成后，模型将被上传到 Amazon S3 里，上传位置由创建 `PyTorch` 对象时提供的 `output_path` 参数指定。

### 模型的验证

您将从 Amazon S3 下载经过训练的模型到笔记本所在实例的本地文件系统，下面的代码将载入模型，然后输入一个随机数，获得推理结果，以图片形式展现出来。


In [ ]:
from helper import *

last_artifact_location = s3_model_artifacts_location + training_job_name

last_model_url = get_object_path_by_filename(last_artifact_location, 'model.tar.gz')
last_output_url = get_object_path_by_filename(last_artifact_location, 'output.tar.gz')

print(last_model_url)
print(last_output_url)

In [ ]:
from sagemaker.s3 import S3Downloader as s3down

!rm -rf ./tmp/* ./model/*
s3down.download(last_model_url, './tmp')
#s3down.download(last_output_url, './tmp')

In [ ]:
!tar -zxf tmp/model.tar.gz -C ./tmp
#!tar -zxf tmp/output.tar.gz -C ./tmp

执行如下指令加载训练好的模型，并通过这个模型产生一组『手写』数字字体。

In [ ]:
import helper
import matplotlib.pyplot as plt
import numpy as np
import torch
from networks.DCGAN.model import Generator
from networks.DCGAN.model_tools import generate_fake_handwriting

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

params = {'nz': hps['nz'], 'nc': hps['nc'], 'ngf': hps['ngf']}
model = helper.load_model("./tmp/generator_state.pth", model_cls=Generator, params=params, device=device)
img = generate_fake_handwriting(model, num_images=64, nz=hps['nz'], device=device)

plt.imshow(np.asarray(img))